Python Code for Ascending and Descending Triad

In [ ]:
import pandas as pd
import numpy as np

def detect_triads(df):
    # Assuming df has columns 'High' and 'Low'
    df['H'] = df['High']
    df['L'] = df['Low']

    # Descending Triad
    df['m1_desc'] = (df['H'] > df['H'].shift(1)) & (df['L'] > df['L'].shift(1))
    df['m2_desc'] = (df['H'] > df['H'].shift(2)) & (df['L'] > df['L'].shift(2)) & (df['H'] > df['H'].shift(1)) & (df['L'] < df['L'].shift(1))
    df['m3_desc'] = (df['H'] > df['H'].shift(3)) & (df['L'] > df['L'].shift(3)) & (df['H'] > df['H'].shift(2)) & (df['L'] < df['L'].shift(2)) & (df['H'] > df['H'].shift(1)) & (df['L'] < df['L'].shift(1))
    df['mark_desc'] = df['m1_desc'] | df['m2_desc'] | df['m3_desc']

    df['value_when_l'] = df.apply(lambda row: df.loc[:row.name, 'L'][df.loc[:row.name, 'mark_desc']].iloc[-1] if any(df.loc[:row.name, 'mark_desc']) else np.nan, axis=1)
    df['value_when_h'] = df.apply(lambda row: df.loc[:row.name, 'H'][df.loc[:row.name, 'mark_desc']].iloc[-1] if any(df.loc[:row.name, 'mark_desc']) else np.nan, axis=1)
    df['highest_since'] = df['H'].rolling(window=len(df), min_periods=1).max()

    df['set_desc'] = (df['L'] < df['value_when_l']) & (df['H'] < df['value_when_h']) & (df['highest_since'].shift(1) < df['value_when_h'])

    df['sig_desc'] = np.where(df['mark_desc'], 1, np.where(df['set_desc'], 0, np.nan))
    df['sig_desc'] = df['sig_desc'].fillna(method='ffill')
    df['cross_desc'] = ((df['sig_desc'].shift(1) > 0.5) & (df['sig_desc'] <= 0.5)).astype(int)

    # Ascending Triad
    df['m1_asc'] = (df['H'] < df['H'].shift(1)) & (df['L'] < df['L'].shift(1))
    df['m2_asc'] = (df['H'] < df['H'].shift(2)) & (df['L'] < df['L'].shift(2)) & (df['H'] > df['H'].shift(1)) & (df['L'] < df['L'].shift(1))
    df['m3_asc'] = (df['H'] < df['H'].shift(3)) & (df['L'] < df['L'].shift(3)) & (df['H'] > df['H'].shift(2)) & (df['L'] < df['L'].shift(2)) & (df['H'] > df['H'].shift(1)) & (df['L'] < df['L'].shift(1))
    df['mark_asc'] = df['m1_asc'] | df['m2_asc'] | df['m3_asc']

    df['value_when_l_asc'] = df.apply(lambda row: df.loc[:row.name, 'L'][df.loc[:row.name, 'mark_asc']].iloc[-1] if any(df.loc[:row.name, 'mark_asc']) else np.nan, axis=1)
    df['value_when_h_asc'] = df.apply(lambda row: df.loc[:row.name, 'H'][df.loc[:row.name, 'mark_asc']].iloc[-1] if any(df.loc[:row.name, 'mark_asc']) else np.nan, axis=1)
    df['lowest_since'] = df['L'].rolling(window=len(df), min_periods=1).min()

    df['set_asc'] = (df['L'] > df['value_when_l_asc']) & (df['H'] > df['value_when_h_asc']) & (df['lowest_since'].shift(1) > df['value_when_l_asc'])

    df['sig_asc'] = np.where(df['mark_asc'], 1, np.where(df['set_asc'], 0, np.nan))
    df['sig_asc'] = df['sig_asc'].fillna(method='ffill')
    df['cross_asc'] = ((df['sig_asc'].shift(1) > 0.5) & (df['sig_asc'] <= 0.5)).astype(int)

    return df

# Usage:
# df = pd.read_csv('your_data.csv')  # Load your data
# df = detect_triads(df)
# Descending Triad signals are in df['cross_desc']
# Ascending Triad signals are in df['cross_asc']

Pinescript code 

In [ ]:
# //@version=5
# indicator("Triad Detection", overlay=true)

# // Custom valuewhen function
# f_valuewhen(_cond, _source, _occurrence) =>
#     v = _source
#     c = 0
#     for i = 1 to 1000
#         if _cond[i]
#             c := c + 1
#             if c == _occurrence
#                 v := _source[i]
#                 break
#     v

# // Custom highest function
# f_highest(_source, _length) =>
#     h = _source
#     for i = 1 to _length
#         h := math.max(h, _source[i])
#     h

# // Custom lowest function
# f_lowest(_source, _length) =>
#     l = _source
#     for i = 1 to _length
#         l := math.min(l, _source[i])
#     l

# // Descending Triad
# m1_desc = high > high[1] and low > low[1]
# m2_desc = high > high[2] and low > low[2] and high > high[1] and low < low[1]
# m3_desc = high > high[3] and low > low[3] and high > high[2] and low < low[2] and high > high[1] and low < low[1]
# mark_desc = m1_desc or m2_desc or m3_desc

# value_when_l_desc = f_valuewhen(mark_desc, low, 1)
# value_when_h_desc = f_valuewhen(mark_desc, high, 1)
# highest_since = f_highest(high, bar_index - bar_index[1] + 1)

# set_desc = low < value_when_l_desc and high < value_when_h_desc and highest_since[1] < value_when_h_desc

# var float sig_desc = na
# sig_desc := mark_desc ? 1.0 : (set_desc ? 0.0 : nz(sig_desc[1]))
# cross_desc = sig_desc[1] > 0.5 and sig_desc <= 0.5

# // Ascending Triad
# m1_asc = high < high[1] and low < low[1]
# m2_asc = high < high[2] and low < low[2] and high > high[1] and low < low[1]
# m3_asc = high < high[3] and low < low[3] and high > high[2] and low < low[2] and high > high[1] and low < low[1]
# mark_asc = m1_asc or m2_asc or m3_asc

# value_when_l_asc = f_valuewhen(mark_asc, low, 1)
# value_when_h_asc = f_valuewhen(mark_asc, high, 1)
# lowest_since = f_lowest(low, bar_index - bar_index[1] + 1)

# set_asc = low > value_when_l_asc and high > value_when_h_asc and lowest_since[1] > value_when_l_asc

# var float sig_asc = na
# sig_asc := mark_asc ? 1.0 : (set_asc ? 0.0 : nz(sig_asc[1]))
# cross_asc = sig_asc[1] > 0.5 and sig_asc <= 0.5

# // Plotting
# plotshape(cross_desc, title="Descending Triad", color=color.red, style=shape.triangledown, location=location.abovebar)
# plotshape(cross_asc, title="Ascending Triad", color=color.green, style=shape.triangleup, location=location.belowbar)

Backtesting Pinescript Triad Update

To use this script:

Replace the indicator() function with strategy() at the beginning of your script.
Copy and paste this new code at the end of your existing Triad Detection script.
Run the script on your chart in Trading View or another platform that supports Pine Script strategies.


Best Pinescript for Triads with Entry Points

In [ ]:
//@version=5
indicator("Triad Detection with Entry Points", overlay=true)

// Custom valuewhen function
f_valuewhen(_cond, _source, _occurrence) =>
    v = _source
    c = 0
    for i = 1 to 1000
        if _cond[i]
            c := c + 1
            if c == _occurrence
                v := _source[i]
                break
    v

// Custom highest function
f_highest(_source, _length) =>
    h = _source
    for i = 1 to _length
        h := math.max(h, _source[i])
    h

// Custom lowest function
f_lowest(_source, _length) =>
    l = _source
    for i = 1 to _length
        l := math.min(l, _source[i])
    l

// Descending Triad
m1_desc = high > high[1] and low > low[1]
m2_desc = high > high[2] and low > low[2] and high > high[1] and low < low[1]
m3_desc = high > high[3] and low > low[3] and high > high[2] and low < low[2] and high > high[1] and low < low[1]
mark_desc = m1_desc or m2_desc or m3_desc
value_when_l_desc = f_valuewhen(mark_desc, low, 1)
value_when_h_desc = f_valuewhen(mark_desc, high, 1)
highest_since = f_highest(high, bar_index - bar_index[1] + 1)
set_desc = low < value_when_l_desc and high < value_when_h_desc and highest_since[1] < value_when_h_desc
var float sig_desc = na
sig_desc := mark_desc ? 1.0 : (set_desc ? 0.0 : nz(sig_desc[1]))
cross_desc = sig_desc[1] > 0.5 and sig_desc <= 0.5

// Ascending Triad
m1_asc = high < high[1] and low < low[1]
m2_asc = high < high[2] and low < low[2] and high > high[1] and low < low[1]
m3_asc = high < high[3] and low < low[3] and high > high[2] and low < low[2] and high > high[1] and low < low[1]
mark_asc = m1_asc or m2_asc or m3_asc
value_when_l_asc = f_valuewhen(mark_asc, low, 1)
value_when_h_asc = f_valuewhen(mark_asc, high, 1)
lowest_since = f_lowest(low, bar_index - bar_index[1] + 1)
set_asc = low > value_when_l_asc and high > value_when_h_asc and lowest_since[1] > value_when_l_asc
var float sig_asc = na
sig_asc := mark_asc ? 1.0 : (set_asc ? 0.0 : nz(sig_asc[1]))
cross_asc = sig_asc[1] > 0.5 and sig_asc <= 0.5

// Entry points
buy_signal = cross_asc
sell_signal = cross_desc

// Plot triad patterns
plotshape(cross_desc, title="Descending Triad", color=color.red, style=shape.triangledown, location=location.abovebar)
plotshape(cross_asc, title="Ascending Triad", color=color.green, style=shape.triangleup, location=location.belowbar)

// Plot entry points
plotshape(buy_signal, title="Buy Signal", color=color.green, style=shape.triangleup, location=location.belowbar, size=size.small)
plotshape(sell_signal, title="Sell Signal", color=color.red, style=shape.triangledown, location=location.abovebar, size=size.small)

// Plot entry prices
plot(buy_signal ? close : na, title="Buy Entry", color=color.green, style=plot.style_circles, linewidth=3)
plot(sell_signal ? close : na, title="Sell Entry", color=color.red, style=plot.style_circles, linewidth=3)

// Alerts
alertcondition(buy_signal, title="Buy Alert", message="Ascending Triad Complete - Buy Signal")
alertcondition(sell_signal, title="Sell Alert", message="Descending Triad Complete - Sell Signal")

Best w/ HTF Detection

In [ ]:
//@version=5
indicator("Simplified HTF Triad Detection", overlay=true)

// Input parameter for HTF
htf_period = input.timeframe("D", "Higher Timeframe for Filter")

// Custom valuewhen function
f_valuewhen(_cond, _source, _occurrence) =>
    v = _source
    c = 0
    for i = 1 to 1000
        if _cond[i]
            c := c + 1
            if c == _occurrence
                v := _source[i]
                break
    v

// Custom highest function
f_highest(_source, _length) =>
    h = _source
    for i = 1 to _length
        h := math.max(h, _source[i])
    h

// Custom lowest function
f_lowest(_source, _length) =>
    l = _source
    for i = 1 to _length
        l := math.min(l, _source[i])
    l

// Descending Triad
m1_desc = high > high[1] and low > low[1]
m2_desc = high > high[2] and low > low[2] and high > high[1] and low < low[1]
m3_desc = high > high[3] and low > low[3] and high > high[2] and low < low[2] and high > high[1] and low < low[1]
mark_desc = m1_desc or m2_desc or m3_desc
value_when_l_desc = f_valuewhen(mark_desc, low, 1)
value_when_h_desc = f_valuewhen(mark_desc, high, 1)
highest_since = f_highest(high, bar_index - bar_index[1] + 1)
set_desc = low < value_when_l_desc and high < value_when_h_desc and highest_since[1] < value_when_h_desc
var float sig_desc = na
sig_desc := mark_desc ? 1.0 : (set_desc ? 0.0 : nz(sig_desc[1]))
cross_desc = sig_desc[1] > 0.5 and sig_desc <= 0.5

// Ascending Triad
m1_asc = high < high[1] and low < low[1]
m2_asc = high < high[2] and low < low[2] and high > high[1] and low < low[1]
m3_asc = high < high[3] and low < low[3] and high > high[2] and low < low[2] and high > high[1] and low < low[1]
mark_asc = m1_asc or m2_asc or m3_asc
value_when_l_asc = f_valuewhen(mark_asc, low, 1)
value_when_h_asc = f_valuewhen(mark_asc, high, 1)
lowest_since = f_lowest(low, bar_index - bar_index[1] + 1)
set_asc = low > value_when_l_asc and high > value_when_h_asc and lowest_since[1] > value_when_l_asc
var float sig_asc = na
sig_asc := mark_asc ? 1.0 : (set_asc ? 0.0 : nz(sig_asc[1]))
cross_asc = sig_asc[1] > 0.5 and sig_asc <= 0.5

// Higher Timeframe Filter
[htf_high, htf_low] = request.security(syminfo.tickerid, htf_period, [high, low])
htf_uptrend = close > htf_low
htf_downtrend = close < htf_high

// Entry signals with HTF filter
buy_signal = cross_asc and htf_uptrend
sell_signal = cross_desc and htf_downtrend

// Plotting
plotshape(cross_desc, title="Descending Triad", color=color.red, style=shape.triangledown, location=location.abovebar)
plotshape(cross_asc, title="Ascending Triad", color=color.green, style=shape.triangleup, location=location.belowbar)

// Plot entry points
plotshape(buy_signal, title="Buy Signal", color=color.green, style=shape.triangleup, location=location.belowbar, size=size.small)
plotshape(sell_signal, title="Sell Signal", color=color.red, style=shape.triangledown, location=location.abovebar, size=size.small)

// Alerts
alertcondition(buy_signal, title="Buy Alert", message="Ascending Triad Complete - Buy Signal")
alertcondition(sell_signal, title="Sell Alert", message="Descending Triad Complete - Sell Signal")